In [7]:
import numpy as np
import pandas as pd
import urllib.request
import folium
from IPython.display import display

appid = "###################################"
api_version = "2.1"
base_url = "http://api.e-stat.go.jp/rest/{api_version}/app/".format(api_version=api_version)
"""
get_type = "getStatsList"
stats_code = "00200521" # 国勢調査
url = base_url + "{get_type}?appId={appid}&statsCode={stats_code}".format(
        api_version=api_version,
        get_type=get_type,
        appid=appid,
        stats_code=stats_code,
        )
print(url) # 確認して取得したいデータのIDを調べる
"""
"""
get_type="getStatsData"
stats_data_id="0003148596" # 最新の調査ID
url = base_url + "{get_type}?appId={appid}&statsDataId={stats_data_id}".format(
        api_version=api_version,
        get_type=get_type,
        appid=appid,
        stats_data_id=stats_data_id
        )
print(url) # 確認して取得したい項目パラメータを調べる
"""

# load data from e-stat api
get_type="getSimpleStatsData"
stats_data_id="0003148596"
cd_cat_01="0000" # 国籍->全て
cd_cat_02="0000" # 性別->男女
cd_cat_03="00710" # 集計地域->全域
lv_area="2" # 集計レベル->都道府県レベル
section_header_flg="2" # セクションヘッダー->無し
url = base_url + "{get_type}?appId={appid}&statsDataId={stats_data_id}&cdCat01={cd_cat_01}&cdCat02={cd_cat_02}&cdCat03={cd_cat_03}&lvArea={lv_area}§ionHeaderFlg={section_header_flg}".format(
        api_version=api_version,
        get_type=get_type,
        appid=appid,
        stats_data_id=stats_data_id,
        cd_cat_01=cd_cat_01,
        cd_cat_02=cd_cat_02,
        cd_cat_03=cd_cat_03,
        lv_area=lv_area,
        section_header_flg=section_header_flg,
        )

d = urllib.request.urlopen(url).read().decode("utf8")
dlines = d.splitlines()[2:]

jcodes = []
names = []
populations = []

for line in dlines:
    line2 = line.replace('"', "").split(",")
    jcode = line2[8]
    name = line2[9]
    population = line2[13]
    jcode = jcode[0:2]
    population = int(population)
    jcodes.append(jcode)
    names.append(name)
    populations.append(population)

df = pd.DataFrame({"jcode" : jcodes, "name" : names, "population" : populations})
display(df)

# create leaflet map by folium
location = [39.702053, 141.15448379999998]
tiles="Stamen Toner"
zoom_start = 5
map = folium.Map(location=location, tiles=tiles, zoom_start=zoom_start)
map.choropleth(
    geo_path="japan.geojson",
    data=df,
    columns=["jcode", "population"],
    key_on="properties.JCODE",
    threshold_scale=[10000, 50000, 100000, 150000, 200000, 300000],
    fill_color="YlGnBu", fill_opacity=0.7, line_opacity=0.2)

display(map)

IndexError: list index out of range